**A PET Scan Simulator**

By: Nicholas Trigger and James Bradley


Using Ideas from:
    Mahoney, D., Huang, S. C., Ricci, A. R., Mazziotta, J. C., Carson, R. E., Hoffman, E. J., & Phelps, M. E. (1987). A Realistic Computer-Simulated Brain Phantom for Evaluation of PET Charactenstics. IEEE transactions on medical imaging, 6(3), 250–257. https://doi.org/10.1109/TMI.1987.4307834

In [ ]:
from ipywidgets import (
    AppLayout,
    FloatSlider,
    IntSlider,
    VBox,
    Output,
    interactive,
    Accordion,
    Checkbox,
    Dropdown,
    IntRangeSlider,
    HBox,
    VBox,
    HTML,
    Layout

)
import matplotlib.pyplot as plt
import numpy as np
from Helpers.image_to_phantom import img_to_phantom, get_image_info
import plotly.graph_objects as go
from PIL import Image
import scipy.io
from scipy import signal
#!pip install scikit-image
from skimage.transform import radon, iradon

# blur slider
blur_slider = FloatSlider(value=5, min=0, max=10, step=0.1, description="Blur:", continuous_update=False)

# Create sliders for Fluence, CT Values and PET Values
fluence_slider = IntSlider(value=10, min=1, max=100, description="Fluence:", continuous_update=False)
c_CSF_slider = FloatSlider(value=1, min=0, max=15, step=0.1, description="CSF Density:", continuous_update=False)
c_WM_slider = FloatSlider(value=2, min=0, max=15, step=0.1, description="WM Density:", continuous_update=False)
c_GM_slider = FloatSlider(value=8, min=0, max=15, step=0.1, description="GM Density:", continuous_update=False)
c_tumor_slider = FloatSlider(value=3, min=0, max=15, step=0.1, description="Tumor Density:", continuous_update=False)

p_CSF_slider = FloatSlider(value=0.2, min=0, max=15, step=0.1, description="CSF Metabolic Activity:", continuous_update=False)
p_WM_slider = FloatSlider(value=2.5, min=0, max=15, step=0.1, description="WM Metabolic Activity:", continuous_update=False)
p_GM_slider = FloatSlider(value=2.7, min=0, max=15, step=0.1, description="GM Metabolic Activity:", continuous_update=False)
p_tumor_slider = FloatSlider(value=6, min=0, max=15, step=0.1, description="Tumor Metabolic Activity:", continuous_update=False)

# Checkbox for tumor presence and tumor location
tumor_check = Checkbox(value=False, description="Tumor Present", disabled=False, indent=False)
tumor_location_x = IntSlider(value=0, min=-10, max=10, step=1, description="Tumor Location X:", continuous_update=False)
tumor_location_y = IntSlider(value=0, min=-10, max=10, step=1, description="Tumor Location Y:", continuous_update=False)
tumor_width_x = FloatSlider(value=5, min=-10, max=10, step=0.1, description="Tumor Width X:", continuous_update=False)
tumor_width_y = FloatSlider(value=5, min=-10, max=10, step=0.1, description="Tumor Width Y:", continuous_update=False)

# Widgets for selecting image and output image
imageInfo = get_image_info()

imageSelector = Dropdown(options=imageInfo.keys(), description="Input Image:")
imageOutputSelect = Dropdown(options=[('Original', 'O'), ('Generated Phantom', 'G'), ('CT', 'C'), ('PET', 'P'), ('CT + PET', 'CP'), ('CT Sinogram', 'CS')], description="Output Image:")

# Output widgets
output = Output() #plot
logs = Output() #logs (print statements)

def run_simulation(fluence, c_CSF, c_WM, c_GM, c_tumor, p_CSF, p_WM, p_GM, p_tumor, blur, tumor_present, tumor_location_x, tumor_location_y, tumor_width_x, tumor_width_y, image, outputType):
    
    img_path = imageInfo[image][0]
    whitematter_color = imageInfo[image][1]
    graymatter_color = imageInfo[image][2]
    
    with logs:
        logs.clear_output(wait=True)
        phantoms, query = img_to_phantom(img_path, image, whitematter_color, graymatter_color, 0, c_WM, c_GM, c_CSF, p_WM, p_GM, p_CSF, tumor_params = [tumor_location_x, tumor_location_y, tumor_width_x, tumor_width_y, c_tumor, p_tumor] if tumor_present else None, tolerance_pct = 8, brain_bound_padding = 10, blur_radius = blur, dbg = False, fileloc="./BrainPhantoms/")
    
    ct_phantom = phantoms[0]
    pet_phantom = phantoms[1]

    ct_sinogram = radon(ct_phantom)
    
    ct_simulation = iradon(ct_sinogram)
    # ADD
    # CODE
    # HERE
    
    with output:
        
        output.clear_output(wait=True)
        match outputType:
            case 'O': # Original
                image = plt.imread(img_path)
                image = np.array(image)
            case 'G': # Generated Phantom
                image = ct_phantom
                image = np.array(image)
            case 'C': # CT Simulation
                image = ct_simulation
                image = np.array(image)
            case 'P':
                pass # PET Simulation
            case 'CP':
                pass # CT + PET Simulation
            case 'CS':
                image = ct_sinogram
                image = np.array(image)
            case _:
                print("Invalid output type selected.")
        
        plt.imshow(image, cmap='gray')
        plt.axis('off')
        plt.tight_layout()
        plt.show()
        
    with logs:
        logs.clear_output(wait=True)
        print(img_path)
        print(query)
        print(outputType)
        
# Create an interactive widget
interactiveWindow = interactive(
    run_simulation,
    fluence=fluence_slider,
    c_CSF = c_CSF_slider,
    c_WM = c_WM_slider,
    c_GM = c_GM_slider,
    c_tumor = c_tumor_slider,
    p_CSF = p_CSF_slider,
    p_WM = p_WM_slider,
    p_GM = p_GM_slider,
    p_tumor = p_tumor_slider,
    blur = blur_slider,
    tumor_present = tumor_check,
    tumor_location_x = tumor_location_x,
    tumor_location_y = tumor_location_y,
    image = imageSelector,
    tumor_width_x = tumor_width_x,
    tumor_width_y = tumor_width_y,
    info = imageInfo,
    outputType=imageOutputSelect
)

# Header and footer
headerMsg = "CT and PET Phantom Generator and Simulator"
header_fmt = HTML(f"<h1 style='margin-top: 10px;margin-bottom: 45px;text-align: center;font-family: sans-serif;font-size: 2.7rem;letter-spacing: 0.15rem;text-transform: uppercase;color: #fff;text-shadow: -4px 4px #ef3550,-8px 8px #f48fb1,-12px 12px #7e57c2,-16px 16px #2196f3,-20px 20px #26c6da,-24px 24px #43a047,-28px 28px #eeff41,-32px 32px #f9a825,-36px 36px #ff5722;'>{headerMsg}</h1>")
spacer = HTML("<hr style='margin: 5px; border: 2px solid black; border-radius: 5px;'>")
header = VBox([header_fmt, spacer])

footerMsg = "By Nick and James"
footer_fmt = HTML(f"<h2 style='margin-top: 25px;margin-bottom: 0px;text-align: center;font-family: sans-serif;font-size: 1rem;letter-spacing: 0.15rem;text-transform: uppercase;color: #000'>{footerMsg}</h2>")
footer = VBox([spacer, footer_fmt])

# Layout the sliders and plot
CTbox = VBox([c_CSF_slider, c_WM_slider, c_GM_slider, c_tumor_slider])
PETbox = VBox([p_CSF_slider, p_WM_slider, p_GM_slider, p_tumor_slider])
TumorBox = VBox([tumor_check, c_tumor_slider, p_tumor_slider, tumor_location_x, tumor_location_y, tumor_width_x, tumor_width_y])

accordion = Accordion(children=[CTbox, PETbox, TumorBox, logs])
accordion.set_title(0, 'CT Values')
accordion.set_title(1, 'PET Values')
accordion.set_title(2, 'Tumor Values')
accordion.set_title(3, 'Logs')

AppLayout(
    header=header,
    right_sidebar=VBox([imageSelector, imageOutputSelect, blur_slider, fluence_slider, accordion], layout=Layout(width='100%', height='100%', align_items='stretch')),
    left_sidebar=None,
    center=VBox([output], layout=Layout(width='75%', height='100%')),
    footer=footer,
    layout=Layout(width='100%', height='100%')
)

AppLayout(children=(VBox(children=(HTML(value="<h1 style='margin-top: 10px;margin-bottom: 45px;text-align: cen…